# Analyzing OpenStreetMap data

In this document I would like to show you a MongoDB queries I used to analyze OpenStreetMap data.

## Import pymogno and acessing database

In [33]:
import pymongo, pprint

client = pymongo.MongoClient('mongodb://localhost:27017/')

db = client.openstreetmap

## Data overview

### Number of valid items

In [34]:
print('Number of valid items: {}'.format(db.elements.count()))

Number of valid items: 52406


In [35]:
def number_of_elements(node_type):
    count_of_type = db.elements.aggregate([
        {'$match': {'type': node_type}},
        {'$group': {'_id': None, 'count': {'$sum': 1}}}
    ])
    
    return list(count_of_type)[0]['count']

### Number of valid nodes

In [36]:
print('Number of valid nodes: {}'.format(number_of_elements('node')))

Number of valid nodes: 46957


### Number of valid ways

In [37]:
print('Number of valid ways: {}'.format(number_of_elements('way')))

Number of valid ways: 5362


### Number of valid relations

In [38]:
print('Number of valid relations: {}'.format(number_of_elements('relation')))

Number of valid relations: 87


In [73]:
def number_of_unique_items(by_field):
    count_of_items = db.elements.aggregate([
        {'$group': {'_id': '${}'.format(by_field), 'count': {'$sum': 1}}},
        {'$group': {'_id': None, 'count': {'$sum': 1}}}
    ])
    
    return list(count_of_items)[0]['count']

### Number of unique authors

In [74]:
print('Number of unique authors: {}'.format(number_of_unique_items('uid')))

Number of unique authors: 642


### The author with the most contributions

### Number of unique changesets

In [76]:
print('Number of unique changesets: {}'.format(number_of_unique_items('changeset')))

Number of unique changesets: 12892
